In [43]:
from time import time
from cv2 import imwrite
import numpy as np
import json
import overpy
import pyproj
from pyproj import CRS

from getData import getData

In [44]:
# Define the bounding box coordinates (min_lon, min_lat, max_lon, max_lat)
bbox = (-6.2610400, 53.3464100, -6.2595700, 53.3474500)

# counting the number of features
features = 0

# Define your Overpass query to retrieve OSM data within the bbox
overpass_query = f"""
[bbox:{bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}];
( 
  way["highway"];
  way["building"];
  way["water"];
  way["landuse"];
  relation["building"];
);
/*added by auto repair*/
(._;>;);
/*end of auto repair*/
out body;
"""

# Initialize the Overpass API client
api = overpy.Overpass()

# Define a mapping of OSM tags to categories
tag_category_mapping = {
    'building': 'Building',
    'highway': 'Road',
    'water': 'Water',
    'landuse': 'Land Use',
    'leisure': 'Leisure',
    'amenity': 'Amenity',
    'natural': 'Natural',
    'railway': 'Railway',
    'power': 'Power',
    # Add more tags and categories as needed
}

# Execute the Overpass query and save the result as a GeoJSON file
osm_data = api.query(overpass_query)

# Convert the result to GeoJSON format
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for way in osm_data.ways:
    # Initialize category to 'Other' as a default
    category = 'Other'

    for tag in way.tags:
        # Check if the tag is in the mapping, and assign the corresponding category
        if tag in tag_category_mapping:
            category = tag_category_mapping[tag]
            features += 1
        print(f"{way.id} -> {category}")

    feature = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [(float(node.lon), float(node.lat)) for node in way.nodes]
        },
        "properties": {
            "category": category
        }
    }
    geojson_data["features"].append(feature)

print(f"Total number of features: {features}")
# Save the result as a GeoJSON file
with open("osm_data.geojson", "w") as f:
    json.dump(geojson_data, f)


25094727 -> Other
25094727 -> Other
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
25094727 -> Road
35078310 -> Other
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
35078310 -> Road
42354359 -> Road
42354359 -> Road
42354359 -> Road
43319604 -> Other
43319604 -> Road
43319604 -> Road
43319604 -> Road
43319604 -> Road
43319604 -> Road
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
76592311 -> Land Use
129212226 -> Road
129212226 -> Road
129212226 -> Road
129212226 -> Road
129212226 -> Road
129212226 -> Roa

In [49]:
# import geopandas as gpd
# gdf = gpd.read_file("osm_data.geojson")
# xmin, ymin, xmax, ymax = gdf.total_bounds
# print(gdf.total_bounds)
# pixel_value = 0.0001
# nodata_value = 0

In [50]:
# category_mapping = {
#     'Building': 1,
#     'Road': 2,
#     'Water': 3,
#     'Land Use': 4,
#     'Leisure': 5,
#     'Amenity': 6,
#     'Natural': 7,
#     'Railway': 8,
#     'Power': 9,
#     'Other': 10
# }
# width = int((xmax - xmin) / pixel_value)
# height = int((ymax - ymin) / pixel_value)
# img = np.zeros((height, width, 1), dtype=np.uint8)
# img.fill(nodata_value)
# imgLandUse = img.copy()

In [51]:
# from rasterio.features import geometry_mask
# from rasterio.transform import from_origin

# for index, row in gdf.iterrows():
#     geom = row['geometry']
#     category = row['category']

#     if category in category_mapping:
#         category_value = str(category_mapping[category])
#     else:
#         category_value = str(10)
    
#     shapes = [(geom, 1)]
#     mask = geometry_mask(shapes, out_shape=(height, width), transform=from_origin(xmin, ymax, pixel_value, pixel_value), invert=True)
#     img[mask] = category_value

#     for y in range(height):
#         for x in range(width):
#             if mask[y][x] == True:
#                 imgLandUse[y][x] = category_value

# # imwrite("osm_data.png", img)

In [52]:
import geopandas as gpd
from rasterio.features import geometry_mask
from rasterio.transform import from_origin

In [54]:
gdf = gpd.read_file("osm_data.geojson")
xmin, ymin, xmax, ymax = gdf.total_bounds
# print(gdf.total_bounds)
pixel_value = 0.00001
nodata_value = 0

category_mapping = {
    'Building': 55,
    'Road': 10,
    'Water': 38,
    'Land Use': 39,
    'Leisure': 5,
    'Amenity': 6,
    'Natural': 7,
    'Railway': 14,
    'Power': 9,
    'Other': 0
}
width = int((xmax - xmin) / pixel_value)
height = int((ymax - ymin) / pixel_value)
print(f"> width: {width}\n> height: {height}")
img = np.zeros((height, width, 1), dtype=np.uint8)
img.fill(nodata_value)


for index, row in gdf.iterrows():
    geom = row['geometry']
    category = row['category']

    if category in category_mapping:
        category_value = category_mapping[category]
    else:
        category_value = 0
    
    shapes = [(geom, 1)]
    mask = geometry_mask(shapes, out_shape=(height, width), transform=from_origin(xmin, ymax, pixel_value, pixel_value), all_touched=True, invert=True)
    img[mask] = category_value

    
    # fill the inside of the polygon
    for y in range(height):
        for x in range(width):
            if mask[y][x] == True:
                img[y][x] = category_value
# save the image
imwrite("osm_data.png", img)

> width: 1094
> height: 450


True

In [58]:
# write img into a text file
with open("osm_data.txt", "w") as f:
    for y in range(height):
        for x in range(width):
            f.write(str(img[y][x][0]) + " ")
        f.write("\n")